In [1]:
using Enzyme
using ArrheniusModel

Test for flow_coefficient


In [4]:

# example from tests
G = [1.0,0.0]
Ea = [0. 0.2; 0.2 0.]
pe = PhaseEnergies(G, Ea)

PhaseEnergies([1.0, 0.0], [0.0 0.2; 0.2 0.0], [0.0 0.2; 1.2 0.0])

In [68]:
# try forward mode without the struct first
# this will give the derivative of each element of the output matrix wrt the 1,2 element of the input
db = Array(zero(pe.barriers))
db[2,1] = 1.0
g12 = Enzyme.autodiff(Forward, arrhenius_rate, Duplicated(pe.barriers, db))[1]
# (convince yourself that this gives the right result)

2×2 Matrix{Float64}:
  0.0          -0.0
 -2.67689e-19   2.67689e-19

In [39]:
realK = arrhenius_rate(pe.barriers)
display(pe.barriers)
display(realK)

2×2 Matrix{Float64}:
 0.0  0.2
 1.2  0.0

2×2 Matrix{Float64}:
 -0.000436534   0.000436534
  6.92002e-21  -6.92002e-21

In [72]:
db = zero(pe.barriers)
dT = 1.0
T = 300.0
K = zero(pe.barriers)
dK = one(pe.barriers)
Enzyme.autodiff(ReverseWithPrimal, arrhenius_rate!, Duplicated(pe.barriers, db), Duplicated(K, dK), Duplicated(T, dT))
db


2×2 Matrix{Float64}:
 0.0          0.0168865
 2.67689e-19  0.0

In [7]:
a = (db, dT)
a[1][1] = 2.0
a


([2.0 0.0; 0.0 0.0], 1.0)

In [8]:

# similarly, to get the derivative wrt the 2,2 element we can do...
db = Array(zero(pe.barriers))
db[2,2] = 1.0
Enzyme.autodiff(Forward, arrhenius_rate, Duplicated(pe.barriers, db))
# (again, convince yourself)

(var"1" = [0.0 -0.0; -0.0 0.0],)

In [3]:
# we can do it wrt the temperature also...
db = Array(zero(pe.barriers))
dT = 1.0
Enzyme.autodiff(Forward, arrhenius_rate, Duplicated(pe.barriers, db), Duplicated(300.0, dT))
# (check this on paper too)

(var"1" = [-1.1257691092137462e-5 1.1257691092137462e-5; 1.0707553411056894e-21 -1.0707553411056894e-21],)

In [11]:
using SciMLSensitivity, OrdinaryDiffEq, Enzyme



In [64]:
function simpletest(du, u, p, t)
    du[1] = u[1] * p[1]
end
p = [1.0]
u0 = [1.0]
prob = ODEProblem(simpletest, u0, (0.0, 1.0), p)
sol = solve(prob, Tsit5(), saveat =0.1)
#display(sol)
function last(u0, p, prob)
    u0 .= solve(prob, Tsit5(), u0 = u0, p = p, saveat = 0.1)[end]
    return nothing
end
display(last(u0, p, prob))
du = zeros(size(u0))
dp = zeros(size(p))
du[1]=1.0 #The output dp will be proportional to this du
#println(sol)
gloss = Enzyme.autodiff(Reverse, last, Duplicated(u0, du), Duplicated(p, dp), Const(prob))
println(dp)

nothing

[7.389054210155433]


In [55]:
function last2(u0, p, prob)
    return solve(prob, Tsit5(), u0 = u0, p = p, saveat = 0.1)[end]
end

function finite_difference_gradient(f, u0, p, prob, epsilon=1e-6)
    grad = zeros(length(p))
    for i in 1:length(p)
        p_forward = copy(p)
        p_backward = copy(p)
        p_forward[i] += epsilon
        p_backward[i] -= epsilon
        a = f(u0, p_forward, prob)
        b = f(u0, p_backward, prob)
        grad[i] = (a[1]-b[1]) / (2 * epsilon)
    end
    return grad
end

finite_difference_gradient (generic function with 2 methods)

In [56]:
# Reset u0 to its initial value
u0 = [1.0]

# Compute the numerical gradient
numerical_grad = finite_difference_gradient(last2, u0, p, prob)
println("Numerical Gradient: ", numerical_grad)

Numerical Gradient: [2.7182806083825284]


In [57]:
println(dp) # this is the square of the gradient

[7.389054210155433]


In [21]:
println(last(u0, p, prob))

19.056274533271502


In [18]:
println(du, dp)

[19.056274299694117][11.396095486944962]


In [13]:
println(du)
println(dp)

[19.056274299694117]
[11.396095486944962]


In [81]:
function fiip(du, u, p, t)
    du[1] = dx = p[1] * u[1] - p[2] * u[1] * u[2]
    du[2] = dy = -p[3] * u[2] + p[4] * u[1] * u[2]
end
p = [1.5, 1.0, 3.0, 1.0];
u0 = [1.0; 1.0];
prob = ODEProblem(fiip, u0, (0.0, 10.0), p)
sol = solve(prob, Tsit5())
display(sol)
loss(u0, p, prob) = sum(solve(prob, Tsit5(), u0 = u0, p = p, saveat = 0.1))
du = zeros(size(u0))
dp = zeros(size(p))
dp[1] = 1.0
println(loss(u0, p, prob))
gloss = Enzyme.autodiff(Reverse, loss, Active, Duplicated(u0, du), Duplicated(p, dp), Const(prob))
println(dp)

retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 34-element Vector{Float64}:
  0.0
  0.0776084743154256
  0.23264513699277584
  0.4291185174543143
  0.6790821987497083
  0.9444046158046306
  1.2674601546021105
  1.6192913303893046
  1.9869754428624007
  2.2640902393538296
  ⋮
  7.584863345264154
  7.978068981329682
  8.48316543760351
  8.719248247740158
  8.949206788834692
  9.200185054623292
  9.438029017301554
  9.711808134779586
 10.0
u: 34-element Vector{Vector{Float64}}:
 [1.0, 1.0]
 [1.0454942346944578, 0.8576684823217128]
 [1.1758715885138271, 0.6394595703175443]
 [1.419680960717083, 0.4569962601282089]
 [1.8767193950080012, 0.3247334292791134]
 [2.588250064553348, 0.26336255535952197]
 [3.860708909220769, 0.2794458098285261]
 [5.750812667710401, 0.522007253793458]
 [6.8149789991301635, 1.9177826328390826]
 [4.392999292571394, 4.1946707928506015]
 ⋮
 [2.6142539677883248, 0.26416945387526314]
 [4.24107612719179, 0.3051236762922018]
 [6.791123785297775

450.6437728782925
[8.349039360098285, -159.3107987785755, 74.93924800431942, -339.3272380777995]


In [17]:
du = zeros(size(u0))
dp = zeros(size(p))
dp[1] = 1.0
gloss = Enzyme.autodiff(Forward, loss, Duplicated(u0, du), Duplicated(p, dp), Const(prob))
gloss[1]


Enzyme.Compiler.EnzymeRuntimeActivityError: Constant memory is stored (or returned) to a differentiable variable.
As a result, Enzyme cannot provably ensure correctness and throws this error.
This might be due to the use of a constant variable as temporary storage for active memory (https://enzyme.mit.edu/julia/stable/faq/#Activity-of-temporary-storage).
If Enzyme should be able to prove this use non-differentable, open an issue!
To work around this issue, either:
 a) rewrite this variable to not be conditionally active (fastest, but requires a code change), or
 b) set Enzyme.API.runtimeActivity!(true) immediately after loading Enzyme (which maintains correctness, but may slightly reduce performance).
Mismatched activity for:   %value_phi = select i1 %.not, {} addrspace(10)* %getfield64, {} addrspace(10)* %getfield, !dbg !205 const val:   %getfield64 = load atomic {} addrspace(10)*, {} addrspace(10)* addrspace(11)* %9 unordered, align 8, !dbg !205
Type tree: {[-1]:Pointer, [-1,0]:Point

In [16]:
dp

4-element Vector{Float64}:
    8.349039360098285
 -159.3107987785755
   74.93924800431942
 -339.3272380777995